# Initialisation

## Imports

In [1]:
import sys, os, csv, json
import logging, importlib

import numpy as np
import matplotlib.pyplot as plt
%matplotlib qt5

%load_ext autoreload
%autoreload 2

sys.path.insert(0, "/home/pleroy/DEV/processing/PoSAR-MC")
import posarmctools.ekfnavtools as ekf
import posarmctools.epsgtools as epsg
import posarmctools.posar as posar
import posarmctools.sbg as sbg
from posarmctools import getNotebookLogger

## Parameters

In [2]:
flight = posar.FlightParams("/home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25_flight.ini")
#flight = posar.FlightParams("/home/pleroy/DATA/PoSAR-X/PIMA-1/flight2.ini")
withPlots = 0
epsg3xxx = epsg.epsg3948
epsgStr = "epsg3948"

## Logger

In [3]:
logger = getNotebookLogger()

## Scene remarkable points

In [4]:
filename = os.path.join(flight.dir_posar, flight.refs)
pts, ptsEpsg, ptsDict = epsg.getReferencePoints(filename, epsg3xxx, file="ini")

In [5]:
np.save(os.path.join(flight.out_dir, "corner_epsg"), ptsEpsg["corner"])
np.save(os.path.join(flight.out_dir, "hangar_epsg"), [ptsEpsg[key] for key in ("h0", "h1", "h2", "h3")])
np.save(os.path.join(flight.out_dir, "runaway_epsg"), [ptsEpsg[key] for key in ("p0", "p1", "p2", "p3")])
np.save(os.path.join(flight.out_dir, "s_epsg"), [ptsEpsg[key] for key in ("s0", "s1", "s2", "s3", "s4", "s5")])
np.save(os.path.join(flight.out_dir, "x_epsg"), [ptsEpsg[key] for key in ("x0", "x1", "x2", "x3")])

# SBG data

## data path definitions

In [6]:
logs = sbg.Logs(flight, epsg3xxx)

load gps
load utc
start ... 0:7:38.060
loop 0 ... 10:16:12.215
stop ... 10:20:18.400
load eventB
load ekf


# PoSAR-MC data

In [7]:
expectedPeriod = 1000000

In [8]:
rec_dates = [f"{flight.day}_" + h for h in flight.hours]
rec_dirs = [os.path.join(flight.dir_posar, flight.day, d) for d in rec_dates]
records = [posar.Record(rec_dir, "record", "bin", utc=logs.utc, period=expectedPeriod, version="X_v1") for rec_dir in rec_dirs]

rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_11_57_59
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_12_01_28
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_12_04_34
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_12_07_40
rec_dir /home/pleroy/DATA/PoSAR-X/PIMA-2/2020_06_25/2020_06_25_12_11_37


In [9]:
validFiles = [record.day_hour for idx, record in enumerate(records) if record.idxTime.size == 0]
notValidFiles = [record.day_hour for idx, record in enumerate(records) if record.idxTime.size > 0]

In [10]:
print(f"*** VALID FILES ***\n{validFiles}\n*** NOT VALID FILES ***\n{notValidFiles})")

*** VALID FILES ***
['2020_06_25_11_57_59', '2020_06_25_12_01_28', '2020_06_25_12_04_34', '2020_06_25_12_07_40', '2020_06_25_12_11_37']
*** NOT VALID FILES ***
[])


# Build record data

In [11]:
record = records[0]

In [12]:
roll_selection = np.interp(record.shiftedLogEvents, logs.euler.timestamps, logs.euler.roll)
pitch_selection = np.interp(record.shiftedLogEvents, logs.euler.timestamps, logs.euler.pitch)
yaw_selection = np.interp(record.shiftedLogEvents, logs.euler.timestamps, logs.euler.yaw)
Lat_Long_Alt = ekf.getInterpolatedGps(record.shiftedLogEvents, logs.gps)
selectedEpsg = epsg.wgs84LongLatToEpsg((Lat_Long_Alt[1], Lat_Long_Alt[0]), epsg3xxx)
Vel_course = ekf.getInterpolatedVelCourse(record.shiftedLogEvents, logs.vel)
Vel_records = ekf.getInterpolatedVel(record.shiftedLogEvents, logs.vel)
x_selection = np.interp(record.shiftedLogEvents, logs.gps.timestamps, logs.gpsEpsg[0])
y_selection = np.interp(record.shiftedLogEvents, logs.gps.timestamps, logs.gpsEpsg[1])
np.save(record.out_dir + "/track_selection_proj", np.stack((x_selection, y_selection), -1))

## Track approximation

In [13]:
JA, JB = [pts[ref] for ref in flight.hours[record.hour]]
xA, yA = ptsEpsg[flight.hours[record.hour][0]]
xB, yB = ptsEpsg[flight.hours[record.hour][1]]
xAB = (xB - xA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5
yAB = (yB - yA) / ( (xB - xA)**2 + (yB - yA)**2 )**0.5
ux = ( xAB, yAB )
uy = ( -yAB, xAB )

# build sceneReferencePoint
JA_epsg = epsg.wgs84LongLatToEpsg((JA[1], JA[0]), epsg3xxx)
JB_epsg = epsg.wgs84LongLatToEpsg((JB[1], JB[0]), epsg3xxx)
JAJB = ( (JA_epsg[0]+JB_epsg[0])/2,
        (JA_epsg[1]+JB_epsg[1])/2 )
shiftInY = 0
sceneReferencePoint = JAJB[0] + shiftInY * uy[0], JAJB[1] + shiftInY * uy[1]

track_model = {"trackModel" : "model(x, p) = p[1]*x + p[2]",
               "ux" : (ux[0], ux[1]),
               "uy" : (uy[0], uy[1]),
               "origX" : sceneReferencePoint[0],
               "origY" : sceneReferencePoint[1]
              }

trackFilename = record.out_dir + "/track_model.json"
with open( trackFilename, 'w' ) as f:
    json.dump( track_model, f )

## Interpolate navigation data for all ramps

In [14]:
## GPS ##
gps_xyz_a, gps_xyz_b = epsg.interpolate_xyz(record, logs.gps, logs.gpsEpsg)
np.save(record.out_dir + "/rampNumber_timeStamp_xyz_gps_a", gps_xyz_a)
np.save(record.out_dir + "/rampNumber_timeStamp_xyz_gps_b", gps_xyz_b)
gps_lla_a, gps_lla_b = epsg.interpolate_lla(record, logs.gps)
np.save(record.out_dir + "/rampNumber_timeStamp_lla_gps_a", gps_lla_a)
np.save(record.out_dir + "/rampNumber_timeStamp_lla_gps_b", gps_lla_b)

## NAV ##
nav_xyz_a, nav_xyz_b = epsg.interpolate_xyz(record, logs.nav, logs.navEpsg)
np.save(record.out_dir + "/n_time_xyz_nav_a", nav_xyz_a)
np.save(record.out_dir + "/n_time_xyz_nav_b", nav_xyz_b)
nav_lla_a, nav_lla_b = epsg.interpolate_lla(record, logs.nav)
np.save(record.out_dir + "/n_time_lla_nav_a", nav_lla_a)
np.save(record.out_dir + "/n_time_lla_nav_b", nav_lla_b)

## EKF ##
rpy_a, rpy_b = logs.euler.interpolate(record)
np.save(record.out_dir + "/n_time_rpy_a", rpy_a)
np.save(record.out_dir + "/n_time_rpy_b", rpy_b)

# Plots

## Compare gps and nav

In [ ]:
plt.figure()
plt.plot(gps_xyz_a[:, 2], gps_xyz_a[:, 3], '.', label="gps")
plt.plot(nav_xyz_a[:, 2], nav_xyz_a[:, 3], label="nav")
plt.plot(selectedEpsg[0], selectedEpsg[1], 'o', label="selectedEpsg")

plt.legend()
plt.grid()
plt.title("x-y")

In [ ]:
plt.figure()
plt.plot(gps_xyz_a[:, 1], gps_xyz_a[:, 4], label="gps_a")
plt.plot(nav_xyz_a[:, 1], nav_xyz_a[:, 4], label="nav_a")
plt.plot(record.shiftedLogEvents, Lat_Long_Alt[2], 'o', label="record.shiftedLogEvents")
plt.plot(record.shiftedLogEvents[0:record.nBins], Lat_Long_Alt[2][0:record.nBins], 
         'or', label="record.shiftedLogEvents [nBins]", markeredgecolor='k')
                                                                  
plt.legend()
plt.grid()
plt.title("z")

## Trajectory

In [15]:
fig, ax = plt.subplots(1,1)

for index, slice_ in enumerate(logs.gps.slices):
    ax.plot(logs.x[slice_], logs.y[slice_], label=logs.gps.loader.hours[index])
    
epsg.addRemarkablePoints(ax, ptsDict, ptsEpsg)
    
ax.plot(selectedEpsg[0], selectedEpsg[1], ".r", markeredgecolor='black', label=record.day_hour)
u = np.sin(Vel_course[1] * np.pi / 180)
v = np.cos(Vel_course[1] * np.pi / 180)
u_yaw = np.sin(yaw_selection)
v_yaw = np.cos(yaw_selection)
ax.quiver(selectedEpsg[0], selectedEpsg[1], u_yaw, v_yaw,
          units='xy', scale=0.02, color='blue',
          width=2, headwidth=3., headlength=4., label='yaw', zorder=10)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid()
ax.legend()
title = epsgStr + " " + logs.conf.day
ax.set_title(title)
ax.set_aspect('equal')
fig.savefig(f"{record.out_dir}/{title}.png", bbox_inches='tight')

In [16]:
fig, ax = plt.subplots(1,1)

epsg.addRemarkablePoint(ax, ptsEpsg['corner'])
    
ax.plot(selectedEpsg[0], selectedEpsg[1], ".r", markeredgecolor='black', label=record.day_hour)
u = np.sin(Vel_course[1] * np.pi / 180)
v = np.cos(Vel_course[1] * np.pi / 180)
u_yaw = np.sin(yaw_selection)
v_yaw = np.cos(yaw_selection)
ax.quiver(selectedEpsg[0], selectedEpsg[1], u_yaw, v_yaw,
          units='xy', scale=0.02, color='blue',
          width=2, headwidth=3., headlength=4., label='yaw', zorder=10)
ax.quiver(selectedEpsg[0], selectedEpsg[1], u, v,
          units='xy', scale=0.02, color='orange',
          width=2, headwidth=3., headlength=4., label='course', zorder=10)
ax.set_xlabel("x")
ax.set_ylabel("y")
ax.grid()
ax.legend()
title = "quiver " + epsgStr + " " + logs.conf.day
ax.set_title(title)
ax.set_aspect('equal')
fig.savefig(f"{record.out_dir}/{title}.png", bbox_inches='tight')

## sbgLogEvents

In [ ]:
delta = 20e6
t0 = record.shiftedLogEvents[0] - delta
t1 = record.shiftedLogEvents[-1] + delta
idx = np.where((logs.eventB.shiftedTimestamps>t0) & (logs.eventB.shiftedTimestamps<t1))

fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)

ax.plot(record.shiftedLogEvents/1e6, record.shiftedLogEvents/1e6,
        "o", markeredgecolor="k", color="w", label=f"PoSAR LogEventB {record.day_hour}")
ax.plot(logs.eventB.shiftedTimestamps[idx]/1e6, logs.eventB.shiftedTimestamps[idx]/1e6,
        '.r', label="SBG LogEventB shiftedTimestamps")
ax.legend()
ax.grid()

ax1.plot(record.shiftedLogEvents[1:]/1e6, np.diff(record.shiftedLogEvents),
         "o", markeredgecolor="k", color="w", label=f"[diff] PoSAR LogEventB {record.day_hour}")
ax1.plot(logs.eventB.shiftedTimestamps[idx][1:]/1e6, np.diff(logs.eventB.shiftedTimestamps[idx]), 
         '.r', label="[diff] SBG LogEventB shiftedTimestamps")
ax1.grid()
ax1.legend()
ax1.set_xlim(t0/1e6, t1/1e6)

fig.suptitle("loopbacks during the records")

In [ ]:
fig, ax = plt.subplots(1, 1)
for slice_ in logs.eventB.slices:
    ax.plot(range(slice_.start, slice_.stop), logs.eventB.data[slice_ ,0], 'x', label="eventB.data")
    ax.plot(range(slice_.start, slice_.stop), logs.eventB.shiftedTimestamps[slice_], '.', label="eventB.shiftedTimestamps")
ax.legend()
ax.grid()

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)

for index, record in enumerate(records):
    ax.plot(record.shiftedLogEvents, record.shiftedLogEvents, '.', label=f"{record.day_hour}")
ax.vlines(logs.utc.timestamps[logs.utc.loopbacks[1:-1]], 0, 2**32 * (len(logs.utc.loopbacks) - 2 + 0.5))
ax.legend()
ax.grid()

for num, slice_ in enumerate(logs.eventB.slices):
    ax1.plot(logs.eventB.shiftedTimestamps[slice_], logs.eventB.shiftedTimestamps[slice_], '.', label=logs.eventB.loader.hours[num])
ax1.vlines(logs.utc.timestamps[logs.utc.loopbacks[1:-1]], 0, 2**32 * (len(logs.utc.loopbacks) - 2 + 0.5))
ax1.legend()
ax1.grid()
fig.suptitle("loopbacks during the records")

## Projection in x-y

In [ ]:
def changeCoordinate(P, O, ux, uy):
    vec = np.c_[P[0] - O[0], P[1] - O[1]].T
    projx = vec[0] * ux[0] + vec[1] * ux[1]
    projy = vec[0] * uy[0] + vec[1] * uy[1]
    return projx, projy

def projectPoint(P, O, ux, uy):
    OP = (P[0] - O[0], P[1] - O[1])
    OP_dot_ux = (OP[0] * ux[0] + OP[1] * ux[1])
    P_prime = (O[0] + OP_dot_ux * ux[0], O[1] + OP_dot_ux * ux[1])
    return P_prime

In [ ]:
C2 = (ptsEpsg['c2'][0], ptsEpsg['c2'][1])
O = projectPoint(C2, sceneReferencePoint, ux, uy)

projx, projy = changeCoordinate((gps_xyz_a[:, 2], gps_xyz_a[:, 3]), O, ux, uy)
dist = np.abs(projx)
minDist = np.amin(dist)
idx = np.where(dist == minDist)
yAtx0 = projy[idx[0][0]]
OASelection = np.c_[x_selection - O[0], y_selection - O[1]].T
projxSelection, projySelection = changeCoordinate((x_selection, y_selection), O, ux, uy)

In [ ]:
fig, (ax, ax1) = plt.subplots(2, 1, sharex=True)
ax.plot(projx, projy, label = f"{record.day_hour} projection, y(0) = {yAtx0:.2f}m")
ax.plot(projxSelection, projySelection,  ".r", markeredgecolor='black')
ax.grid()
ax.legend()
ax.set_xlabel('x [m]')
ax.set_ylabel('y [m]')
ax.set_title('différence entre la trajectoire consigne et la trajectoire mesurée')

ax1.plot(projx, gps_xyz_a[:, 4], label = f"{record.day_hour} z")
ax1.grid()
ax1.legend()
ax1.set_xlabel('x [m]')
ax1.set_ylabel('z [m]')

title = "track projection" + logs.conf.day
fig.savefig(f"{record.out_dir}/{title}.png", bbox_inches='tight')

In [ ]:
plt.figure()
plt.plot(gps_xyz_a[1:, 2], gps_xyz_a[1:, 3])
plt.plot(C2[0], C2[1], 'D', label='C2')
plt.plot(O[0], O[1], 'o', label='O')
plt.legend()
plt.grid()

## Euler

In [17]:
fig, (ax0, ax1, ax2) = plt.subplots(3, 1, sharex=True)

ax0.plot(rpy_a[:, 1], rpy_a[:, 2] * 180 / np.pi, 'o', label='roll (interp)')
ax0.plot(logs.euler.timestamps, logs.euler.roll * 180 / np.pi, '.-', label='roll (logs)')
ax0.legend()
ax0.grid()

ax1.plot(rpy_a[:, 1], rpy_a[:, 3] * 180 / np.pi, 'o', label='pitch (interp)')
ax1.plot(logs.euler.timestamps, logs.euler.pitch * 180 / np.pi, '.-', label='pitch (logs)')
ax1.legend()
ax1.grid()

ax2.plot(rpy_a[:, 1], rpy_a[:, 4] * 180 / np.pi, 'o', label='yaw (interp)')
ax2.plot(logs.euler.timestamps, logs.euler.yaw * 180 / np.pi, '.-', label='yaw (logs)')
ax2.legend()
ax2.grid()

In [ ]:
a = np.array([[0, 1, 2], [7, 8, 9]])
b = np.array([[3, 4, 5], [10, 11, 12]])

In [ ]:
a

In [ ]:
np.c_[a, b]